# Working with FITS Tables and Images for Spectral Data

## Author
[Dhanesh Krishnarao (DK)](http://www.astronomy.dk)

## Learning Goals
* Find interesting source coordinates using `astroquery`
* Read and plot a FITS image and overplot source points using `wcsaxes`
* Read spectral data from a FITS table
* match queried coordinates to observations in a FITS table
* Write out a new FITS table with newly calculated entries

## Keywords
FITS-files, FITS-images, FITS-tables, astroquery, coordinates, units, spectroscopy

In the tutorial we will read and write FITS tables, read and a visualize 2D FITS image, find sources using astroquery, match coordinates, and convert observational data to different units by making some assumptions. 

This tutorial will walk you through a simple analysis of ionized gas observed towards some high energy sources, calculating some column densities of diffuse HII gas from spectroscopic observations of H-Alpha emssion. The analysis will be saved as a FITS table. 

The primary libraries we will be using are: [astroquery](http://www.astropy.org/astroquery/), [matplotlib](https://matplotlib.org/), [specutils](https://specutils.readthedocs.io/en/latest/)

They can be installed using conda:
- `conda install -c astropy astroquery`
- `conda install -c astropy specutils`

Alternatively, if you don't use conda, you can use pip.

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.utils.data import download_file
from astropy.wcs import WCS
from astropy.io import fits
from astropy.table import Table
from astropy.nddata import StdDevUncertainty
from astropy.visualization.wcsaxes.frame import EllipticalFrame
from astropy.constants import Ryd

from astroquery.vizier import Vizier

from specutils import Spectrum1D

%matplotlib inline

# Set tutorial path for data files
tutorial_path = ''

# Find HII regions

We will be looking for known HII regions in the Milky Way from the WISE catalog of HII regions [(Anderson+, 2014)](http://adsabs.harvard.edu/abs/2014ApJS..212....1A). We want to look at observed H-Alpha spectra from the [WHAM Sky Survey]{http://www.astro.wisc.edu/wham-site/wham-sky-survey/wham-ss} and estimate some column densities of ionized gas along the line of sight towards these HII regions. 

The WISE HII region catalog is available on vizier under the name [J/ApJS/212/1/wisecat](http://vizier.cfa.harvard.edu/viz-bin/VizieR-3?-source=J/ApJS/212/1/wisecat). We can use astroquery to peek at this catalog, and query based on our requirements. 

## Use astroquery to retrieve a list of HII regions with relevant information

In [ ]:
catalog_name = 'J/ApJS/212/1/wisecat'

#Query the catalog and get a sample look to see what information we can see
catalog_sample_list = Vizier.get_catalogs(catalog = catalog_name)

print(catalog_sample_list)

`catalog_sample_list` is now an astropy TableList, containing just 1 table corresponding to the WISE catalog. It appears to have 16 columns of infomration. We can view this sample astropy Table by selecting it from the TableList

In [ ]:
catalog_sample = catalog_sample_list[0]

catalog_sample

Some useful columns for us are going to Cl, which tells us some classification information about the source, GLON and GLAT, the Galactic Coordinates of the source, and Dist, the distance to the HII region if known. 

For this exercise, we only want sources that have a distance estimate provided and will limit ourselves to soures that are known HII regions (Cl = 'K'). Details of what these columns mean can be found on the [VizieR catalog website](http://vizier.cfa.harvard.edu/viz-bin/VizieR-3?-source=J/ApJS/212/1/wisecat).

<div class="alert alert-info">
Note: by default, `Vizier` will only return the first 50 entries, hence the number of rows being 50. We should manually lift this restriction so that we can get the full list from here on out. 

This can be done using
`
Vizier.ROW_LIMIT = -1
`
to lift the limit entire. Alternatively, you can set a specific larger size if you want avoid accidently downloading too much data!
</div>

Now that we know the column names we want to filter by, we can use `.query_constraints` method to get our select catalog. The column names we found above can be used as additional arguments to filter through the catalog. Here, we want to select targets with provided distance estimates (or Dist > 0.0) and targets where their classification is known (Cl = K)

In [ ]:
#Lift the row limit for the astroquery Vizier class
Vizier.ROW_LIMIT = -1

#Query the catalog and save the astropy Table of infomration
hii_regions = Vizier.query_constraints(catalog = catalog_name, Cl = 'K', Dist = '>0.0')[0]

hii_regions

We now have our catalog of 828 WISE HII regions. There's a lot of information in this table that we don't necessarily need, so lets simplify our lives and delete some columns. We only need to have the name (WISE), the coordinates (GLON and GLAT), and the distance (Dist). We'll keep the velocity information as well to help compare with our WHAM spectra later as well. 

Each column can be referenced by using its name.

In [ ]:
#Delete extra columns of information we don't need
del(hii_regions["Cl"], hii_regions['Rad'], hii_regions['Ref'], hii_regions['Name'], hii_regions['GLONs'], 
    hii_regions['GLATs'], hii_regions['Vm1'], hii_regions['Mol'], hii_regions['_RA.icrs'], hii_regions['_DE.icrs'])

hii_regions

astropy Tables can also store coordinates directly as an `astropy.SkyCoord` object, which can make catalog matching and converting between different coordinate frames very simple. We can add a new column with a combined coordinate entry and get rid of the two separate columns for `GLON` and `GLAT`

When working with data from an astropy Table, we can also extract the data as a regular numpy array, or as an astropy Quantity, which contains unit information. This can be done using the `.quantity` method. 

In [ ]:
#Create SkyCoord objects of the coordinates
hii_coord = SkyCoord(hii_regions['GLON'].quantity, hii_regions['GLAT'].quantity, frame = 'galactic')

#Enter the SkyCoord infomration as a new column to our Table
hii_regions['Coord'] = hii_coord

#Remove the now extraneous GLON and GLAT columns
del(hii_regions['GLON'], hii_regions['GLAT'])

hii_regions

<div class="alert alert-info">
Note: If you wanted to convert your coordinates to ICRS RA and Dec, you can do so with one line of code:
`
hii_regions['Coord'].icrs
`
</div>

# WHAM Sky Survey Map

Now that we have our list of HII regions from WISE, we can take a look at our spectroscopic data from WHAM. 

## Download and visualize the WHAM all-sky map

An integrate FITS image of this survey is available online at [http://www.astro.wisc.edu/wham/ss/wham-ss-DR1-v161116-170912-int-grid.fits](http://www.astro.wisc.edu/wham/ss/wham-ss-DR1-v161116-170912-int-grid.fits). We can download this file directly and plot it, overlaying our identified HII region positions. 

For ease of use - this file is also available through this tutorial directly, so we can download the file from there.


In [ ]:
'''
# Download the wham integrated FITS image from the UW-Madison host

wham_image_file = download_file(
    'http://www.astro.wisc.edu/wham/ss/wham-ss-DR1-v161116-170912-int-grid.fits',
    cache=True, show_progress = True)
'''

#Download the wham integrated FITS image
wham_image_file = download_file(tutorialpath + 
                                'wham-ss-DR1-v161116-170912-int-grid.fits',
                                cache=True, show_progress = True)

In [ ]:
# Open the FITS file for reading
wham_image_hdu = fits.open(wham_image_file)

# Display some basic information on the FITS image file
wham_image_hdu.info()

This shows us some basic info about the FITS image file. Most importantly, the 0-th element of the HDU object contains our primary image which has dimensions of (1440 x 720). We need two pieces of infomration from this file to fully visualize it - the data array and the header. We can use the header to get additional information about the data and to build a WCS object to convert pixel positions in the image to real world coordinates. Once we access the data and header, we can close the FITS file to avoid using up extra memory. 

In [ ]:
# Access and store the data and header
wham_image_data = wham_image_hdu[0].data
wham_image_header = wham_image_hdu[0].header

# Close the file
wham_image_hdu.close()

# Create a WCS object from the header information
wham_image_wcs = WCS(wham_image_header)

# Print out the header infomration
wham_image_header

FITS headers can contain a lot of useful infomration about the data you are working with, ranging for citation information, to data specific clarifications that can be vital to proper analysis. 

Astropy is pretty great about extracting relevent information to make visualizing this data straightforward. We will use `matplotlib` combined with `wcsaxes` to display this all-sky map and visually inspect where our select HII regions appear. 

This is done primarily when initializing an axis object where we can specifiy a projection using our WCS object. We'll also select a Log based colorscale using the `norm` argument in `imshow` and `LogNorm()` from `matplotlib`.

In [ ]:
# Initiate a matplotlib figure object
fig = plt.figure(figsize = (18,12))
ax = fig.add_subplot(111, projection = wham_image_wcs)

# Create an image object on our wcsaxis
im = ax.imshow(wham_image_data, norm = LogNorm())

# Make the coordinate grid show up in white
ax.coords.grid(color='white')

# Clip the image to the frame
im.set_clip_path(ax.coords.frame.patch)

Pretty good for a first attempt, but let's spice it up a little!

First off, we're making a map of the entire sky - this means there are lots of distortions happening at the edges of this image. We can make this look a little nicer by changing the `frame_class` of our `wcsaxis` object to be an `EllipticalFrame` instead of the default `RectangularFrame`. This is done by adding an extra argument to the axis initiation line, something along the lines of:

`
ax = fig.add_subplot(111, projection = wham_image_wcs, frame_class = EllipticalFrame)
`

We can also change the colormap, and add some axis labels, chose a min and max intensity with `vmin` and `vmax`, and include a colorbar.

We can also overplot points using `ax.scatter` by specifying a coordinate transform to convert our catalog longitude and latitude to pixel positions on the image. An example of this line would be:

`
ax.scatter(hii_regions['Coord'].l, hii_regions['Coord'].b, transform=ax.get_transform('world'))
`
<a id='ex1'></a>
### Exercise 1: Make it pretty!
Try this out as an exercise and sow how 'pretty' you can make your map!

In [ ]:
'''
Temporarily have solution here for notebook testing
'''

# Initiate a matplotlib figure object
fig = plt.figure(figsize = (18,12))
ax = fig.add_subplot(111, projection = wham_image_wcs, frame_class = EllipticalFrame)

# Create an image object on our wcsaxis
im = ax.imshow(wham_image_data, norm = LogNorm(), cmap = 'Reds', vmin = 0.1, vmax = 100)

# Make the coordinate grid show up in white
ax.coords.grid(color='white')

# Add axis labels
ax.set_ylabel("Galactic Latitude (degrees)", fontsize = 16)

# Add a colorbar
cbar = plt.colorbar(im, pad = .07, orientation = 'horizontal')
cbar.set_label('Intensity (R)', size = 16)

# Overplot points from the HII Region Catalog
ax.scatter(hii_regions['Coord'].l, hii_regions['Coord'].b, transform=ax.get_transform('world'), 
           c = 'Blue', s = 15, alpha = .5)


# Clip the image to the frame
im.set_clip_path(ax.coords.frame.patch)

Looks like most of these HII regions are confined near the Galactic Plane - not a surprise since the WISE catalog only surveys near the plane!

# WHAM Kinematic Survey Data

We can now take a look at a more complete version of the [WHAM Sky Survey](http://www.astro.wisc.edu/wham-site/wham-sky-survey/wham-ss) data in the form of ~50,000 individual spectra. This can give us an extra dimension of information to analyze these HII regions and the surrouding gas. 

## Examine the WHAM Kinematic Data

The data is available as a FITS Table and can be downloaded directly from [http://www.astro.wisc.edu/wham/ss/wham-ss-DR1-v161116-170912.fits](http://www.astro.wisc.edu/wham/ss/wham-ss-DR1-v161116-170912.fits) and read in as an astropy Table. 

Once again for ease of use, this file is also available through this tutorial so we can download by just referring to the filename `wham-ss-DR1-v161116-170912.fits` with `tutorial_path + 'wham-ss-DR1-v161116-170912.fits'

<a id='ex2'></a>
### Exercise 2: Download the survey file to cache open it to display some info

When doing this exercise - name the opened FITS file HDU as 
`
wham_data_hdu
`
so that the rest of the tutorial will work properly.

In [ ]:
'''
Temporarily have solution here for notebook testing
'''

# Download the Kinematic Data Table
wham_data_file = download_file(tutorial_path + 
                               'wham-ss-DR1-v161116-170912.fits',
                               cache=True, show_progress = True)

# Open the FITS file for reading
wham_data_hdu = fits.open(wham_data_file)

In [ ]:
# Display some basic information on the FITS image file
wham_data_hdu.info()

This FITS File is a little different than the last one we dealt with. Here, we have two HDU entries. One is a BinTable - what we will read in as an astropy Table, and another just holds some other general information. Each of these HDU entries have their own headers, both are useful so let's take a closer look!

In [ ]:
# View the Primary Header
wham_data_hdu[0].header

In [ ]:
# View the Table Header
wham_data_hdu[1].header

We can get some more specific information about this table through the `.columns` attribute, showing us the data format, units, and column names.

In [ ]:
wham_data_hdu[1].columns

Reading in this data as an astropy Table can be done with just a single line of code!

In [ ]:
#Read in data as a Table
wham_data = Table(wham_data_hdu[1].data)

#close the FITS file
wham_data_hdu.close()

wham_data

Just like our queried table of HII regions from the WISE catalog, we have another astropy Table with a load of spectroscopic data from the WHAM survey looking H-Alpha emission. 

Unfortunately, the units of these columns didn't seem to propogate through to the Table immediately, so we can manually enter these in. Remember that the header we looked at above printed out the proper units for each column so we can lock those units into these data before we make any changes!

We can also create a new column for a combined SkyCoord entry instead of individual GLON and GLAT entries like we did with the HII region catalog.

In [ ]:
wham_data['GAL-LON'].unit = u.deg
wham_data['GAL-LAT'].unit = u.deg
wham_data['VELOCITY'].unit = u.km / u.s
wham_data['DATA'].unit = u.R / (u.km / u.s)
wham_data['VARIANCE'].unit = (u.R / (u.km / u.s))**2
wham_data['INTEN'].unit = u.R
wham_data['ERROR'].unit = u.R
wham_data['OINTEN'].unit = u.R

wham_data['COORD'] = SkyCoord(wham_data['GAL-LON'], wham_data['GAL-LAT'], frame = 'galactic')

del(wham_data['GAL-LON'], wham_data['GAL-LAT'])

In [ ]:
wham_data

Wow - 49,640 spectra is a lot! We only need a small subset of these spectra that correspond to our HII regions we selected for the WISE catalog. 

Notice how some columns, such as VELOCITY or DATA are actually arrays with 151 elements - astropy Tables can hold a lot of information in a very organized way!

The astropy SkyCoord class allows us to easily match coordinates in catalogs using the 
`.match_to_catalog_sky()` 
method. This method can take an input SkyCoord object (either a single coordinate, or many) and match them up to a different SkyCoord array specified in the argument. 
This method returns three results: the matched catalog indices, the angular (2D) distance between the matched coordinates, and (if possible) the physical (3D) distance between the matched coordinates. 

<div class="alert alert-info">
Note: The 3D distance is only provided if additional information beyond an angular coordinate is included in the SkyCoord Object. This is not the case with these data, but you can read more about SkyCoord and this method [here](http://docs.astropy.org/en/stable/coordinates/matchsep.html).

Since we don't need this third array fromt he result, we can just assign it to a placeholder `_` so that it doesn't use up memory.

</div>

We have a (relatively) small list of HII regions for which we want to find matching WHAM spectra. So we will use the WHAM kinematic data SkyCoord object in argument of the `.match_to_catalog_sky()` method acting on the HII region SkyCoord object. 

<div class="alert alert-warning">
Remember that you can access the SkyCoord object from the astropy Tables directly!
</div> 

In [ ]:
# Match our catalogs
match_idx, match2d, _ = hii_regions['Coord'].match_to_catalog_sky(wham_data['COORD'])

# Make a quick histogram of the 2D angular separations
sep_hist = plt.hist(match2d, bins = 'auto')

WHAM has an angular beam size of 1 degree, so our histograms confirms that our catalog has been matched well - no angular separations over ~0.5 degrees!

Let's make a new Table, containing all of the information found in the `hii_regions` Table we made earlier and some select WHAM kinematic data and separations.

In [ ]:
# Create a sub Table of the WHAM matched data
matched_wham_data = wham_data[match_idx]
matched_wham_data

<div class="alert alert-info">
Note: This new Table has some duplicate rows because some of our HII regions fall within a single WHAM beam.

</div>

In [ ]:
# Construct a new Table from the two existing Tables
wham_hii_regions = Table(hii_regions.columns['WISE', 'Vi1', 'DV1', 'Dist', 'Coord'])

# Add in WHAM data
wham_hii_regions['WHAM_Coord'] = matched_wham_data['COORD']
wham_hii_regions['Velocity'] = matched_wham_data['VELOCITY']
wham_hii_regions['Data'] = matched_wham_data['DATA']
wham_hii_regions['Variance'] = matched_wham_data['VARIANCE']
wham_hii_regions['Intensity'] = matched_wham_data['INTEN']
wham_hii_regions['Error'] = matched_wham_data['ERROR']
wham_hii_regions['Offset'] = match2d

# Rename the Coord column to make it more clear
wham_hii_regions.rename_column('Coord', 'WISE_Coord')

wham_hii_regions

We now have a handy astropy Table with all the information we want about some known HII regions and observed H-Alpha emssion surrounding it. 

# Analyze Spectra and Calculate Results

Now that our data is all organized, we can dive into to science!

Astropy simplifies this process with `units` to do calculations quickly.

<div class="alert alert-info">
Probably Won't use specutils actually.... at least for now...

</div>

In the future, the [specutils](https://specutils.readthedocs.io/en/latest/) package will make it even easier to analyze spectroscopic data! For now, we will just use as a handy wrapper to hold our data for now. 

In particular, we will focus on the `Spectrum1D` class from specutils, which provides a convenient framework for dealing with spectroscopic data. 

`Spectrum1D` asks for at least two general pieces of information: a Flux and Spectral Axis. 
For us, we will be using a velocity as our Spectral Axis, so we will also want to specify a `rest_value` so that conversions from between velocity, wavelength, and frequency can be made on the fly. 

We will also include uncertainties associcated with the spectra directly into the Spectrum1D object we build. 

## Investigate the last entry in our Table

For this example, let's take a look at the last entry in our Table.

In [ ]:
# Calculate H-Alpha rest wavelength
BalmerAlpha = 1. / (Ryd * (1 / 2.**2 - 1 / 3.**2))

# Get some infomration from the table about the last entry
entry = -1
wham_c = wham_hii_regions['WHAM_Coord'][entry]
wise_c = wham_hii_regions['WISE_Coord'][entry]
wham_intensity = wham_hii_regions['Intensity'].quantity[entry]
wham_error = wham_hii_regions['Error'].quantity[entry]
offset = wham_hii_regions['Offset'].quantity[entry]
hii_region_distance = wham_hii_regions['Dist'].quantity[entry]

# Create a Spectrum1D object
spec = Spectrum1D(wham_hii_regions['Data'].quantity[entry], spectral_axis = wham_hii_regions['Velocity'].quantity[entry], 
                  uncertainty = StdDevUncertainty(np.sqrt(wham_hii_regions['Variance'].quantity[entry])), 
                  rest_value = BalmerAlpha, velocity_convention = 'optical')

# Plot a quick Spectrum
plt.plot(spec.velocity, spec.flux)

Now that we've taken a quick look at an entry from our Table, let's think about how we can get some useful physical measurements from the WHAM spectra!

Following some assumptions, such as a constant temperature of Diffuse Ionized Gas of $8000$ K, an H-Alpha intensity measured in units of Rayleighs (R or `u.R`) can be converted to an Emission Measure (EM) with the conversion scaling of $1$R is equivalent to an EM of $2.25~cm^{-6}~pc$, where

$$
EM = \int n_e^2~dl
$$

$n_e$ is the electron density or ionized gas density, and $dl$ integrates along the line of sight. 

If we assume a constant/smooth distribution of ionized gas, you can convert an Emission Measure to an estimate of the column density of ionized gas following

$$
N_e = \int n_e~dl = n_e~D = \left( \frac{EM}{D} \right)^{\frac{1}{2}}~D = EM^{\frac{1}{2}}~D^{\frac{1}{2}}
$$

where D is the line of sight distance.

## Convert H-Alpha Intensity to HII Column Density

Let's create the framework to find an HII column density along sightlines to our HII regions. 
We will need to assume that all the H-Alpha emission we have observed is from between us and the HII region and evenly distributed in order to do this. 

`astropy.units` allows us to do this easily and convert our final result to our desired cgs base units!

In [ ]:
# Function to convert H-Alpha intensity to an Emission Measure
def ha_to_em(ha_intensity):
    return ha_intensity * 2.25 * (u.cm)**(-6.) * u.pc / u.R

# Function to convert Emission Measure and Distance to an HII Column Density
def em_to_column_density(em, distance):
    return np.sqrt(em * distance).decompose(u.cgs.bases)

# Combined Function for H-Alpha Intensity and Distance to HII Column Dnesity
def ha_to_column_density(ha_intensity, distance):
    return em_to_column_density(ha_to_em(ha_intensity), distance)

# Calculate for our selected Table Entry
emission_measure = ha_to_em(wham_intensity)
hii_column = ha_to_column_density(wham_intensity, hii_region_distance)

hii_column

<a id='ex3'></a>
### Exercise 3: New Table Columns
#### Add two new columns to the `wham_hii_regions` astropy Table that contains the calculated Emission Measure and HII Column Density

To keep the rest of the notebook working properly, name these new columns 'EM' and 'N_HII'

In [ ]:
'''
Temporary solution here for notebook testing
'''
# Emission Measure column
wham_hii_regions['EM'] = ha_to_em(wham_hii_regions['Intensity'].quantity)

# Column Density column
wham_hii_regions['N_HII'] = ha_to_column_density(wham_hii_regions['Intensity'].quantity, 
                                              wham_hii_regions['Dist'].quantity)

In [ ]:
# Display our final Table
wham_hii_regions

## Save our Table to a File

We have all this information - we should save it so we can easily access it again, or share it with collaborators!
`astropy.Table` allows you to write out a Table to file in many format by specifying a `format = ''` such as 
`
format = 'ascii'
format = 'fits'
format = 'hdf5'
format = 'latex'
`

The `'latex'` option can be especially useful if you want to make a Table of data that's ready for publication. Here, let's try writing out our file as a simple ascii .txt file. We can also specify the delimiter to use for separating our columns.

In [ ]:
# Write data to file 
wham_hii_regions.write('wham_hii_regions.txt', format = 'ascii', delimiter = '|')

## Challenge:

### Propogation of Errors
Create new columns to our Table that properly calculate errors associated with our newly calculated Emission Measure and HII Column Density. 

### Write out a subset of the Table to latex
Create a `latex` style table ready for publication with select columns from our Table. Try to format the output to be a little cleaner using the `formats={}` dictionary as an argument in the `.write()` method. You can read more documentation on this method [here](http://docs.astropy.org/en/stable/io/unified.html#built-in-readers-writers).

# Possible Exercise Solutions

## [Exercise 1:](#ex1)

In [ ]:
# Initiate a matplotlib figure object
fig = plt.figure(figsize = (18,12))
ax = fig.add_subplot(111, projection = wham_image_wcs, frame_class = EllipticalFrame)

# Create an image object on our wcsaxis
im = ax.imshow(wham_image_data, norm = LogNorm(), cmap = 'Reds', vmin = 0.1, vmax = 100)

# Make the coordinate grid show up in white
ax.coords.grid(color='white')

# Add axis labels
ax.set_ylabel("Galactic Latitude (degrees)", fontsize = 16)

# Add a colorbar
cbar = plt.colorbar(im, pad = .07, orientation = 'horizontal')
cbar.set_label('Intensity (R)', size = 16)

# Overplot points from the HII Region Catalog
ax.scatter(hii_regions['Coord'].l, hii_regions['Coord'].b, transform=ax.get_transform('world'), 
           c = 'Blue', s = 15, alpha = .5)


# Clip the image to the frame
im.set_clip_path(ax.coords.frame.patch)

## [Exercise 2:](#ex2)

In [ ]:
# Download the Kinematic Data Table
wham_data_file = download_file(tutorial_path + 
                               'wham-ss-DR1-v161116-170912.fits',
                               cache=True, show_progress = True)

# Open the FITS file for reading
wham_data_hdu = fits.open(wham_data_file)

## [Exercise 3:](#ex3)

In [ ]:
# Emission Measure column
wham_hii_regions['EM'] = ha_to_em(wham_hii_regions['Intensity'].quantity)

# Column Density column
wham_hii_regions['N_HII'] = ha_to_column_density(wham_hii_regions['Intensity'].quantity, 
                                              wham_hii_regions['Dist'].quantity)